In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from sklearn.metrics import f1_score, accuracy_score
from tqdm import tqdm
import random
import numpy as np

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def set_seed(seed_value=42):
    """Set seed for reproducibility."""
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)

    # If you are using GPU
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

# Call the function at the start of your script
set_seed(42)

In [3]:
# Hyperparameters
BATCH_SIZE = 8
INPUT_SIZE = 640
EPOCHS = 30
LEARNING_RATE = 0.0005
PATIENCE = 5
WARMUP_EPOCHS = 3
T_MAX = 10

In [4]:
# Data Transforms
transform = transforms.Compose([
    transforms.Resize((INPUT_SIZE, INPUT_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [5]:
# Datasets and DataLoaders
train_dataset = datasets.ImageFolder('/kaggle/input/nitec-multiclass-dataset/DATASET/train', transform=transform)
val_dataset = datasets.ImageFolder('/kaggle/input/nitec-multiclass-dataset/DATASET/test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers = 2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers = 2)

In [6]:
def yolo(model_name):
    # Model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = torch.hub.load('pytorch/vision:v0.13.0', model_name, weights='IMAGENET1K_V1')
    num_ftrs = model.classifier[1].in_features
    model.classifier[1] = nn.Linear(num_ftrs, 5)
    model = model.to(device)

    # Loss and Optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay = 0.0005)

    # Learning Rate Scheduler with Warmup
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=T_MAX)

    # Training Function
    def train(epoch):
        model.train()
        total_loss = 0.0
        for data, target in tqdm(train_loader, desc=f"Epoch {epoch}"):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch}: Training Loss: {total_loss/len(train_loader)}")

    # Validation Function
    def validate():
        model.eval()
        val_loss = 0.0
        all_preds = []
        all_targets = []
        for data, target in tqdm(val_loader, desc="Validating"):
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            val_loss += loss.item()
            preds = output.argmax(dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(target.cpu().numpy())

        f1 = f1_score(all_targets, all_preds, average='macro')
        acc = accuracy_score(all_targets, all_preds)
        print(f"Validation Loss: {val_loss/len(val_loader)} | F1 Score: {f1} | Accuracy: {acc}")
        return f1

    # Main Training Loop with Early Stopping
    best_f1 = 0.0
    counter = 0
    for epoch in range(1, EPOCHS+1):
        if epoch <= WARMUP_EPOCHS:
            warmup_factor = epoch / WARMUP_EPOCHS
            for param_group in optimizer.param_groups:
                param_group['lr'] = LEARNING_RATE * warmup_factor
        train(epoch)
        torch.cuda.empty_cache()
        with torch.no_grad():
          current_f1 = validate()
        scheduler.step()

        # Save best model
        if current_f1 > best_f1:
            best_f1 = current_f1
            torch.save(model.state_dict(), f'{model_name}_best_model_at_epoch_{epoch}.pth')
            counter = 0
        else:
            counter += 1
            if counter >= PATIENCE:
                print("Early stopping triggered!")
                break

    print(f"Training Completed for {model_name}!")



In [7]:
models = ['efficientnet_b0','efficientnet_b1','efficientnet_b2']
for model_name in models:
    torch.cuda.empty_cache()
    yolo(model_name)

Downloading: "https://github.com/pytorch/vision/zipball/v0.13.0" to /root/.cache/torch/hub/v0.13.0.zip
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-3dd342df.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-3dd342df.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 155MB/s]
Epoch 1: 100%|██████████| 741/741 [03:14<00:00,  3.82it/s]


Epoch 1: Training Loss: 0.3857674545076815


Validating: 100%|██████████| 83/83 [00:15<00:00,  5.29it/s]


Validation Loss: 0.2675470807806993 | F1 Score: 0.7179707341858746 | Accuracy: 0.9092284417549168


Epoch 2: 100%|██████████| 741/741 [03:07<00:00,  3.95it/s]


...

Validation Loss: 0.3005236325955933 | F1 Score: 0.7991227655286248 | Accuracy: 0.913767019667171


Epoch 17: 100%|██████████| 741/741 [04:18<00:00,  2.87it/s]


Epoch 17: Training Loss: 0.14939983426162776


Validating: 100%|██████████| 83/83 [00:16<00:00,  4.91it/s]

Validation Loss: 0.3403527650505961 | F1 Score: 0.7945081486485392 | Accuracy: 0.9092284417549168
Early stopping triggered!
Training Completed for efficientnet_b2!


In [8]:
# import os
# import pandas as pd
# from torchvision.io import read_image
# from torchvision.transforms.functional import to_pil_image

# def submit(model_name):
#     model = torch.hub.load('pytorch/vision:v0.13.0', model_name, weights='IMAGENET1K_V1')
#     model = model.to(device)
    
#     # Load the trained model
#     model.load_state_dict(torch.load(f'{model_name}_best_model.pth'))
#     model.eval()

#     # Inference
#     file_indices = []
#     classes = []

#     test_folder = '/kaggle/input/datasaur-test/test'
#     test_images = [f for f in os.listdir(test_folder) if f.endswith('.jpeg')]

#     for image_name in tqdm(test_images, desc="Inference"):
#         image_path = os.path.join(test_folder, image_name)

#         # Load image and preprocess
#         image = to_pil_image(read_image(image_path))
#         image = transform(image).unsqueeze(0).to(device)

#         # Predict
#         with torch.no_grad():
#             output = model(image)
#             pred = output.argmax(dim=1).item()

#         # Store results
#         file_index = image_name.split('.jpeg')[0]
#         file_indices.append(file_index)
#         classes.append(pred)

#     # Create DataFrame and save to CSV
#     submission_df = pd.DataFrame({
#         'file_index': file_indices,
#         'class': classes
#     })

#     submission_df.to_csv(f'{model_name}_submission.csv', index=False)
#     print("Saved predictions to submission.csv")


In [9]:
# for model_name in models:
#     torch.cuda.empty_cache()
#     submit(model_name)